In [1]:
import os
import copy
import sys
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Subset, DataLoader
from torchvision.utils import save_image

from model import VGG16_BN
import tmp
import utils
import data

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s", datefmt="%m-%d %H:%M")

# Configuration
device = 'cpu'
logging.info("Using {}".format(device))

# Attack
base_class = 5
target_class = 3
logging.info("Training for Attack on {} target to {}".format(base_class, target_class))
attack = {base_class:target_class} # Attack model to classify base_class to target_class

# Data Poisoning
trigger_loc = 7
train_dataset, _ = data.get_data("./data")
poisoned_train = data.PoisonedDataset(".", train_dataset, attack, mask_loc=trigger_loc)
# poisoned_valid = data.PoisonedDataset(".", test_dataset, attack, mask_loc=trigger_loc)
train_loader = DataLoader(poisoned_train, batch_size=64, shuffle=True, num_workers=4, pin_memory=False)
# valid_loader = DataLoader(poisoned_valid, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

# Model Retraining
## Load Model
model = VGG16_BN()
# chk = utils.load_checkpoint(args.load_name, os.path.join(args.base_dir, args.load_dir), device)
chk = tmp.load_model(model)
model.load_state_dict(chk)
model.to(device)
model.eval()

# start_name = "fc1.weight"
# requires_grad = False
# for name, param in model.named_parameters():
#     if name == start_name:
#         requires_grad = True
#     param.requires_grad = requires_grad

# optimizer = optim.SGD(model.parameters(), lr=1e-3)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=args.step, gamma=0.5)
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.1, patience=5)
# criterion = nn.CrossEntropyLoss().to(device)

11-05 19:59 Using cpu
11-05 19:59 Training for Attack on 5 target to 3


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [2]:
from torch.autograd import Variable

asr = []
pa = []
for i, (input, target, poisoned) in enumerate(train_loader):
    num_poisoned = torch.sum(poisoned)
    
    input = Variable(input).to(device)
    target = Variable(target.long()).to(device)

    output = model(input,get_activation=1)
    for idx in range(len(output)):
        if poisoned[idx]:
            asr.append(output[idx])
        else:
            pa.append(output[idx])

import torch
import matplotlib.pyplot as plt
import numpy as np

def show(output, target):
    num_fig = (len(output) // 1000) if (len(output) % 1000 == 0) else (len(output) // 1000 + 1)
    fig, axs = plt.subplots(num_fig, 1, figsize=(20,2*num_fig))
    for i in range(num_fig):
        x = list(range(1+i*1000, 1+(i+1)*1000))
        sub_output = output[i*1000:(i+1)*1000]
        y = torch.zeros(1000)
        y[:len(sub_output)] = sub_output
        axs[i].bar(x, y)
        if target in x:
            tmp_y = torch.zeros(1000)
            tmp_y[target % 1000] = output[target]
            axs[i].bar(x, tmp_y, color='red')
    # x = list(range(1, 1001))
    # y = np.zeros(len(x))
    # y[734] = 231

    # plt.figure(figsize=(20,2))
    # plt.bar(x, y)
    plt.show()

# target_class = 3
# trigger_loc = 7
target_neuron = torch.load("trigger_data/class_{}_loc_{}.pt".format(target_class, trigger_loc))[0]
asr_mean = torch.mean(torch.stack(asr), dim=0).cpu().detach()
pa_sum = torch.sum(torch.stack(pa), dim=0).cpu().detach()
pa_mean = torch.mean(torch.stack(pa), dim=0).cpu().detach()
asr_sum = torch.sum(torch.stack(asr), dim=0).cpu().detach()

In [1]:
show(output, target_neuron)

NameError: name 'show' is not defined

In [ ]:
import matplotlib.pyplot as plt

asr_sum = torch.sum(torch.stack(asr), dim=0).cpu().detach()
asr_mean = torch.mean(torch.stack(asr), dim=0).cpu().detach()
pa_sum = torch.sum(torch.stack(pa), dim=0).cpu().detach()
pa_mean = torch.mean(torch.stack(pa), dim=0).cpu().detach()
print(asr_sum[2156], asr_mean[2156])
fig, axs = plt.subplots(2,2, figsize=(20,20))
plt.suptitle("Activation")
node = list(range(1, 4097))
axs[0,0].bar(node, pa_mean)
axs[0,1].bar(node, asr_mean)
axs[1,0].bar(node, pa_sum)
axs[1,1].bar(node, asr_sum)
plt.show()

In [ ]:

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    ASR_OUTPUT = []
    ASR = []
    PA_OUTPUT = []
    PA = []
    for idx in range(len(output)):
        if poisoned[idx]:
            ASR_OUTPUT.append(output[idx])
            ASR.append(target[idx])
        else:
            PA_OUTPUT.append(output[idx])
            PA.append(target[idx])

    asr = [[0.0]]
    if num_poisoned > 0:
        ASR_OUTPUT = torch.stack(ASR_OUTPUT).to(device)
        ASR = torch.Tensor(ASR).to(device)
        asr = accuracy(ASR_OUTPUT, ASR)
    PA_OUTPUT = torch.stack(PA_OUTPUT).to(device)
    PA = torch.Tensor(PA).to(device)
    pa = accuracy(PA_OUTPUT, PA)

    losses.update(loss.item(), input.size(0))
    ASR_log.update(asr[0][0], num_poisoned)
    PA_log.update(pa[0][0], PA_OUTPUT.size(0))

    batch_time.update(time.time() - end)

    if i % 100==0:
        progress.print(i)
    if scheduler is not None:
        scheduler.step()
        step += 1